<a href="https://colab.research.google.com/github/Alex-Ukraine/ml-selfedu/blob/main/CNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pooling (изменение масштаба)
* MaxPooling - отбор наибольших значений;
* MinPooling - отбор наименьших значений;
* AveragePooling - отбор средних значений;

Ян Лекун, Convolutional Neural Networks (CNN)
Сверточные нейронные сети
Есть понятия ядро фильтра и свёртка

слой свёточной нейронной сети задаётся 
* keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), ...)

* filters - число ядер(каналов);
* kernel_size - размер ядра (в виде котежа двух чисел);
* strides - шас сканирования фильтров по осям плоскости (по умолчанию один пиксел)

пример архитиктуры:
> model = keras.Sequential([Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28,1)),
    ...
])

дальше мы должны укрупнять масштаб полученных признаков:
> keras.layers.MaxPooling2D(pool_size=(2,2), strides=None, padding='valid', data_format=None)

> pool_size - размер окна

> strides - шаг сканирования по координатам плоскости

> model = keras.Sequential([Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28,1)),
                          MaxPooling2D((2,2), strides=2)])

> Conv1D, Conv2D, Conv3D
* аудио - 1D(одномерный сигнал)
* изображения - 2D(двумерный сигнал)
* видео - 3D(трехмерный сигнал)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # 60 000 изображений в обучающей выборке и 10 000 в тестовой
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
%matplotlib inline

Загрузка обучающей и тестовой выборок

* x_train - изображения цыфр обучающей выборки;
* y_train - вектор соответствуюющих значений цифр (например, если на i-м изображении нарисована 5, то y_train[i]=5);
* x_test - изображения цифр тестовой выборки;
* y_test - вектор соответствующих значений цифр для тестовой выборки.

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() # загружаем данные в коллекции

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
# стандартизация входных данных
x_train = x_train / 255
x_test = x_test / 255

Преобразование входных данных значений в векторы по категориям

In [4]:
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

In [5]:
x_train = np.expand_dims(x_train, axis=3) # добавляем 4-ю ось
x_test = np.expand_dims(x_test, axis=3)

Формирование модели НС и вывод её структуры в консоль

In [6]:
model = keras.Sequential([
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

print(model.summary()) # вывод структуры НС в консоль

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               401536    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

Компиляция НС с оптимизацией по Adam и критерием - категориальная кросс-энтропия

In [7]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy', # любимчик публики задачи классификации
              metrics=['accuracy'])

Запуск процесса обучения: 80% - обучающая выборка, 20% - выборка валидации

In [8]:
# validation_split - разбиение на обучающую выборку и проверочную
his = model.fit(x_train, y_train_cat, batch_size=32, epochs=5, validation_split=0.2) 

Epoch 1/6
1500/1500 [==============================] - 61s 41ms/step - loss: 0.1420 - accuracy: 0.9564 - val_loss: 0.0567 - val_accuracy: 0.9838
Epoch 2/6
1500/1500 [==============================] - 63s 42ms/step - loss: 0.0476 - accuracy: 0.9856 - val_loss: 0.0490 - val_accuracy: 0.9859
Epoch 3/6
1500/1500 [==============================] - 60s 40ms/step - loss: 0.0311 - accuracy: 0.9902 - val_loss: 0.0420 - val_accuracy: 0.9884
Epoch 4/6
1500/1500 [==============================] - 60s 40ms/step - loss: 0.0234 - accuracy: 0.9926 - val_loss: 0.0500 - val_accuracy: 0.9863
Epoch 5/6
1500/1500 [==============================] - 60s 40ms/step - loss: 0.0169 - accuracy: 0.9948 - val_loss: 0.0432 - val_accuracy: 0.9864
Epoch 6/6
1500/1500 [==============================] - 60s 40ms/step - loss: 0.0130 - accuracy: 0.9954 - val_loss: 0.0378 - val_accuracy: 0.9896


получим loss и accuracy

In [9]:
model.evaluate(x_test, y_test_cat)

313/313 [==============================] - 4s 12ms/step - loss: 0.0326 - accuracy: 0.9908


[0.03263330087065697, 0.9908000230789185]